# Import dependencies

In [16]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import subprocess
from sqlalchemy import create_engine

from config import PGEND_POINT
from config import PGDATABASE_NAME
from config import PGUSER_NAME
from config import PGPASSWORD

# AWS RDS Connection

In [4]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [5]:
conn, cursor = connect()

Connected!


# SQL Queries

### Create earthquakes table

In [6]:
# Creating the earthquakes table
query_earthquakes = sql.SQL("""CREATE TABLE earthquakes (
  latitude DECIMAL,
  longitude DECIMAL,
  magnitude DECIMAL,
  event_date DATE
);""")

In [7]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_earthquakes)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()

Transaction rolled back due to error: relation "earthquakes" already exists



### Create injectionVolumes table

In [36]:
# Creating the injectionVolumes table
query_injectionVolumes = sql.SQL("""CREATE TABLE injection_volumes (
  api_number INT,
  surface_longitude DECIMAL,
  surface_latitude DECIMAL,
  injection_date DATE,
  injection_end_date DATE,
  volume_injected_bbls DECIMAL
);""")

In [37]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_injectionVolumes)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction committed successfully!


### Create pressureData table

In [10]:
# Creating the pressureData table
query_pressureData = sql.SQL("""CREATE TABLE pressure_data (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL
);""")

In [11]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction rolled back due to error: relation "pressure_data" already exists



### Create query_pressureData_13 table

In [ ]:
# Creating the query_pressureData_13 table
query_pressureData_13 = sql.SQL("""CREATE TABLE pressure_data_13 (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [ ]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_13)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()

### Create query_pressureData_revised table

In [ ]:
# Creating the query_pressureData_revised table
query_pressureData_revised = sql.SQL("""CREATE TABLE pressure_data_revised (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [ ]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_revised)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()

### Define functions and variables required to load data into the tables

In [17]:
def connect():
    # Set up a connection to the postgres server.
    conn_string = f"postgresql+psycopg2://{PGUSER_NAME}:{PGPASSWORD}@{PGEND_POINT}:5432/{PGDATABASE_NAME}"
    engine = create_engine(conn_string)
    print("Connected to the database!")
    return engine

In [18]:
def load_data_to_table(dataframe, table_name, engine):
    dataframe.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print(f"Data loaded into {table_name} successfully!")


In [28]:
# Connect to the database
engine = connect()

Connected to the database!


In [20]:
# Create databases from the csv files
earthquakes_df = pd.read_csv('earthquake-data/earthquakes.csv')
injection_volumes_df = pd.read_csv('injectionVolumes-Data/injectionVolumes.csv')
pressure_data_df = pd.read_csv('pressure-data/Updated_Pressure_Data_with_LatLon.csv')
pressure_data_13_df = pd.read_csv('pressure-data/delta_pressure_layer13.csv')
pressure_data_revised_df = pd.read_csv('pressure-data/delta_pressure_all_layers.csv')

### Load earthquakes data

In [25]:
# Rename earthquakes columns to match the table
earthquakes_df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude', 'Magnitude': 'magnitude', 'Event_Date': 'event_date'}, inplace=True)

In [29]:
# Load the earthquakes data
load_data_to_table(earthquakes_df, 'earthquakes', engine)

Data loaded into earthquakes successfully!


### Load injectionVolumes data

In [31]:
# Rename injectionVolumes columns to match the table
injection_volumes_df.rename(columns={'API Number': 'api_number', 'Surface Longitude': 'surface_longitude', 'Surface Latitude': 'surface_latitude', 'Injection Date': 'injection_date', 'Injection End Date': 'injection_end_date',
                                     'Volume Injected: BBLs': 'volume_injected_bbls'}, inplace=True)

In [38]:
# Load the injectionVolumes data
load_data_to_table(injection_volumes_df, 'injection_volumes', engine)

Data loaded into injection_volumes successfully!


### Load pressure data

In [40]:
# Rename pressureData columns to match the table
pressure_data_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude'}, inplace=True)

In [42]:
# Load the pressure data
load_data_to_table(pressure_data_df, 'pressure_data', engine)

Data loaded into pressure_data successfully!


### Load pressure_data_13

In [ ]:
# Rename pressure_data_13 columns to match the table
pressure_data_13_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

In [ ]:
# Load the pressure_data_13 data
load_data_to_table(pressure_data_13_df, 'pressure_data_13', engine)

### Load pressure data for all layers

In [ ]:
# Rename pressureData columns to match the table
pressure_data_revised_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

In [ ]:
# Load the pressure data
load_data_to_table(pressure_data_revised_df, 'pressure_data_revised', engine)